In [ ]:
import os
import sys
import time
from naoqi import ALProxy
import threading
import qi
import numpy as np 

In [ ]:
# NAO_IP = "10.0.255.22"
# NAO_PORT = 9559
PEPPER_IP = "10.0.255.8"
PEPPER_PORT = 9559

In [ ]:
# tts = ALProxy("ALTextToSpeech", PEPPER_IP, PEPPER_PORT)
# tts.say("Hello, world!")

In [ ]:
CAMERA_DISPARITY = 10
FOCAL_LENGTH = 30

# defining the camera coordinates 
C1 = (0,0,0)
C2 = (3,5,6) # completely random

Getting pixel to read world mapping (will be done once)

In [ ]:
def calc_scale(img_c1, REAL_DISTANCE, c1_x, c1_y, c2_x, c2_y):
  # default resolution taken = 1280x960
  
  width, height = img_c1.size

  # Calculate the center coordinates
  center_x = width / 2
  center_y = height / 2

  camera_disparity = (c2_x - c1_x, c2_y - c1_y)
  scale = camera_disparity/REAL_DISTANCE
  # scale contains info only about x and y 
  return scale

In [ ]:
def object_center(img):

def get_object_disparity(img, obj_x, obj_y):
  # resopnsible for X_L, X_R
  # Get the width and height of the image in pixels
  width, height = img.size

  # Calculate the center coordinates
  center_x = width / 2
  center_y = height / 2

  obj_x, obj_y = object_center(img) # copy ninad's code or use YOLO to detect the object and find the center 
  object_disparity = (obj_x - center_x, obj_y - center_y)
  return object_disparity


def calc_depth(CAMERA_DISPARITY, FOCAL_LENGTH, left_img, right_img):
  X_L = get_object_disparity(left_img)
  mod_X_L = np.sqrt(X_L[0]^2 + X_L[1]^2)
  X_R = get_object_disparity(right_img)
  mod_X_R = np.sqrt(X_R[0]^2 + X_R[1]^2)
  depth = FOCAL_LENGTH/(1-(CAMERA_DISPARITY/(CAMERA_DISPARITY-(mod_X_L + mod_X_R))))
  return depth

def get_world_coordinates(scale, c1_x, c1_y, z_origin, depth):
  world_coordinate = (scale[0] * c1_x, scale[1] * c1_y, z_origin + depth)
  return world_coordinate

In [ ]:
c1_x, c1_y = object_center(img_c1)
c2_x, c2_y = object_center(img_c2)
REAL_DISTANCE = 50
scale = calc_scale(img_c1, img_c2, REAL_DISTANCE)

In [ ]:
# you may take new sets of images now 
# c1_x, c1_y = object_center(img_c1)
# c2_x, c2_y = object_center(img_c2)

depth = calc_depth(CAMERA_DISPARITY, FOCAL_LENGTH, img_c1, img_c2)


# here the z_origin is 0, as the camera1's world coordinate is (0,0,0)
world_coordinate = get_world_coordinates(scale, c1_x, c1_y, 0, depth)
